In [ ]:
import json
from src.cu_estimator import CuEstimator
from pandas.io.json import json_normalize
import pandas as pd
import numpy as np

In [ ]:
def load_df(path: str):
    with open(path) as f:
        data = json.load(f)
    df = json_normalize(data)
    df = df.rename(columns=lambda col: col.rpartition('.')[-1])
    return df

In [ ]:
df_prod = load_df('../data/1/previous_heats_with_properties.json')
df_prod.head()

In [ ]:
with open('../data/1/constraints.json') as f:
    data = json.load(f)['scrap_type_constraints_per_heat']
df_cons = pd.DataFrame(data)
df_cons.head()

In [ ]:
df_orders = load_df('../data/1/scrap_orders.json')
df_orders['price_per_kg'] = df_orders['price_per_ton'] / 1000
prices = df_orders.groupby('scrap_type')['price_per_kg'].mean()
prices = prices.rename({'bushling':'bushlings'})
prices

---

### Build a system of linear equations and solve its simplex method.
**Constrains:**<br>
 * C(x) = cost function -> min
 * bush + pig iron + shread + skulls = heat_weight
 * additional constrains from constraints.json
 
 `C(x) = bush*prices[bush] + pig_iron*prices[pig_iron] + ...`

In [ ]:
scraps = ['bushlings', 'pig_iron', 'municipal_shred', 'skulls']

Cost function -> minimize

In [ ]:
w = np.array([prices[scrap] for scrap in scraps])
w

Bushlings + Pig Iron + Shred + Skulls = Heat weight

In [ ]:
def get_mass_constraints(expected_mass):
    return [-1]*len(scraps), -expected_mass

In [ ]:
def get_additional_constraints(df: pd.DataFrame):
    def parse_constrain(row: pd.Series):        
        cons_a = [0]*len(scraps)
        
        idx = scraps.index(row['scrap_type'])
        cons_a[idx] = -1 # constrain for the scrap type
                           
        multiplier = -1 if row['type']=='minimum' else 1 
        cons_b =  multiplier * row['weight']
        return cons_a, cons_b

    return (parse_constrain(row) for _, row in df.iterrows())

Combain all the constrains together

In [ ]:
a_ub = []
b_ub = []
a_eq = []
b_eq = []

# optimaze the first record
row = df_prod.iloc[0]

In [ ]:
mass_a, mass_b = get_mass_constraints(row['required_weight'])

# the mass should be as required
a_eq.append(mass_a)
b_eq.append(mass_b)

In [ ]:
# the scrap amount should be <= additional limits
for cons_a, cons_b in get_additional_constraints(df_cons):
    a_ub.append(cons_a)
    b_ub.append(cons_b)

In [ ]:
from scipy.optimize import linprog

res = linprog(w, A_eq=a_eq, b_eq=b_eq, A_ub=a_ub, b_ub=b_ub, bounds=(0, None))

In [ ]:
res